# Building and organizing complex queries

* capitalisation and whitespace
* If a select statement has more than one column, put each on a new line, indented from the select statement.
* Always capitalize SQL function names and keywords
* Put each clause of your query on a new line.
* Use indenting to make subqueries appear logically separate.

## Names subqueries

One way to alleviate this is to use a with clause. With clauses allow you to define one or more named subqueries before the start of the main query. The main query then refers to the subquery by it's alias name, just as if it's a table in the database.

```SQL
WITH [alias_name] AS ([subquery])

SELECT [main_query]
```


Defining multiple Subqueries in a WITH block

```SQL
WITH
    [alias_name] AS ([subquery]),
    [alias_name_2] AS ([subquery_2]),
    [alias_name_3] AS ([subquery_3]),

SELECT [main_query]
```

While each subquery can be independent, we can actually use the result of the first subquery in subsequent subqueries, and so on. This can be a useful way of building readable complex queries.

Let's look at a simple example where we create three named subqueries that build on each other.


```SQL
WITH
    usa AS
        (
        SELECT * FROM customer
        WHERE country = "USA"
        ),
    last_name_g AS
        (
         SELECT * FROM usa
         WHERE last_name LIKE "G%"
        ),
    state_ca AS
        (
        SELECT * FROM last_name_g
        WHERE state = "CA"
        )

SELECT
    first_name,
    last_name,
    country,
    state
FROM state_ca
```


## Views

It would be nice to permanently define a subquery that we can use again and again.

We do this by creating a view, which we can then use in all future queries. An easy way to think of this is the WITH clause creates a temporary view. The syntax for creating a view is:

```SQL
CREATE VIEW database.view_name AS
    SELECT * FROM database.table;
```


If we wish to redefine a view, we first have to delete, or drop the existing view:


DROP VIEW chinook.customer_2;


# UNION

[select_statement_one]
UNION
[select_statement_two]

Rather than using the ON keyword, the statements before and after UNION must have the same number of columns, with compatible types in order. We'll learn more about types in a later mission, but at as an example, FLOAT and INT are compatible types, but FLOAT and TEXT are not).
    


```SQL
    WITH joined_query AS (
SELECT * FROM customer_usa
INTERSECT
SELECT * FROM customer_gt_90_dollars
)

SELECT 
   e.first_name  || " " || e.last_name employee_name,
   COUNT(jq.customer_id) customers_usa_gt_90
   FROM employee e
   LEFT JOIN joined_query jq ON e.employee_id = jq.support_rep_id 
   WHERE e.title = "Sales Support Agent"
   GROUP BY employee_name
   ORDER BY employee_name
```




# Table relations and normation

## SQLite3 shell

To launch the SQLite shell, you use the sqlite3 command followed by the name of the database file as an argument.

```bash
/home/dq$ sqlite3 chinook.db
```

## dot commands

### .headers on
The first thing you may notice is that we don't have the column names displayed. SQLite has a number of dot commands which you can use to help you work with databases. When you use a dot command, you don't need to use a semicolon. One that you'll want to use often is .headers on, which switches column headers on

### .mode

There are several other dot commands you'll use often:

.help - Displays help text showing all dot commands and their function.
.tables - Displays a list of all tables and views in the current database.
.shell [command] - Run a command like ls or clear in the system shell.
.quit - Quits the SQLite shell


# Data types in SQLite

Each column in SQLite must have a type. While some database systems have as many as 50 distinct data types, SQLite uses only 5 behind the scenes:

TEXT
INTEGER
REAL
NUMERIC
BLOB

You can also use the SQLite dot command .schema [table_name] to view the schema for a table you have just created to check where you might have gone wrong.

## Creating a new db

To practice, we'll create a new table in a new database file. If you launch the SQLite shell with the argument of a filename that doesn't exist, SQLite will create an empty database with that filename.

sqlite> CREATE TABLE user (user_id INTEGER, first_name TEXT, last_name TEXT)    
   ...> ;                                                                       
sqlite> .schema user                                                            
CREATE TABLE user (user_id INTEGER, first_name TEXT, last_name TEXT);           
sqlite> .quit   


## Primary and FOreign keys 

We previously learned that each table has one or more columns shaded in yellow, which indicates they are the primary key. A primary key is a unique identifier for each row - you cannot have two rows in a table with the same value for the primary key column(s).


This is known as a foreign key. By defining a foreign key, our database engine will prevent us from adding rows where the foreign key value doesn't exist in the other table, which helps to prevent errors in our data (note that by default SQLite doesn't force foreign key constraints, however we have changed the default for this mission).

### Defining a primary key
Usually, a primary key is specified as part of a create statement. Once the primary key is defined, the database engine will prevent any new rows from being added to the database if they have the same primary key as any existing rows. If we wanted to re-create the table from the previous exercise with a primary key, we would use this syntax:

```SQL
CREATE TABLE user (
    user_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);
```

### Defining a foreign key

Let's say we wanted to create a new table purchase which tracks basic information about a purchase made by one of our users. Our create statement might look like this:

```SQL
CREATE TABLE purchase (
    purchase_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    purchase_date TEXT,
    total NUMERIC,
    FOREIGN KEY (user_id) REFERENCES user(user_id)
);
```


# Database Normalisation

The process of optimizing the design of databases to minimize issues like
* Data Duplication
* Data Integrity
* Data Modification


When two or more columns combine to form a primary key it is called a compound primary key. To create a compound primary key, you use the PRIMARY KEY clause:

```SQL
CREATE TABLE [table_name] (
    [column_one_name] [column_one_type],
    [column_two_name] [column_two_type],
    [column_three_name] [column_three_type],
    [column_four_name] [column_four_type],
    PRIMARY KEY (column_one_name, column_two_name)
);
```

# INSERT DATA

INSERT INTO TABLE
VALUES
(1,2,3),
(2,3,4);

# Altering a table

A better approach would be to add a column that has a boolean value to show whether the row is active or not, and just change that value if the user wants to delete a track. We can do a similar thing with the wishlists themselves, so users can delete (or technically, deactivate) wishlists they no longer want to use.

We'll need to add a column to each of our tables. We can use the ALTER statement to do this.

```SQL
ALTER TABLE [table_name]
ADD COLUMN [column_name] [column_type];
```

# Update table data

```SQL
UPDATE [table_name]
SET [column_name] = [expression]
WHERE [expression]
```


The WHERE clause is optional, and can contain any expression that would be valid in a SELECT statement.

There are several variations we can use for our SET clause. First we can use a single value:

```SQL
UPDATE customer
SET phone = "+55 (12) 3921-4464"
WHERE customer_id = 1
```

We can use a subquery that returns a single value:

```SQL
UPDATE track
SET unit_price = (
                    SELECT AVG(unit_price)
                    FROM track
                 )
```



We can use a column, or function on an existing column:

```SQL
UPDATE track
SET unit_price = unit_price * 1.1
```


Lastly, we can set more than one column at once:


```SQL
UPDATE wishlist_track
SET
    active = 1,
    purchased = 0;
UPDATE wishlist_track
SET
    active = 1,
    purchased = 0;
    ```